## Simulation of  modified Stuart-Landau (MSL) model with (generalized) phase model

### Setup

In [1]:
using GeneralizedPhaseModel
using DifferentialEquations, PyPlot
using PyCall
axes_grid1 = pyimport("mpl_toolkits.axes_grid1")

PyObject <module 'mpl_toolkits.axes_grid1' from 'C:\\ProgramData\\Miniconda3\\lib\\site-packages\\mpl_toolkits\\axes_grid1\\__init__.py'>

In [2]:
plt.rc("text", usetex=true)
PyCall.PyDict(plt."rcParams")["font.size"] = 14
PyCall.PyDict(plt."rcParams")["xtick.direction"] = "in"
PyCall.PyDict(plt."rcParams")["ytick.direction"] = "in"
PyCall.PyDict(plt."rcParams")["xtick.minor.visible"] = true
PyCall.PyDict(plt."rcParams")["ytick.minor.visible"] = true
PyCall.PyDict(plt."rcParams")["xtick.top"] = true 
PyCall.PyDict(plt."rcParams")["ytick.right"] = true 
PyCall.PyDict(plt."rcParams")["font.family"] = "Arial"
PyCall.PyDict(plt."rcParams")["text.latex.preamble"] = [raw"\usepackage{amsmath}"];

sys:1: MatplotlibDeprecationWarning: Support for setting the 'text.latex.preamble' or 'pgf.preamble' rcParam to a list of strings is deprecated since 3.3 and will be removed two minor releases later; set it to a single string instead.


### Model definition

In [3]:
# Parameters and variables
c = 0.7; d = 0.8; eps = 0.08; # FHN parameters

# FHN vector field
dvdt(X, I) = X[1] - X[1]^3/3.0 - X[2] + I + 0.875
dudt(X) = eps * (X[1] + c - d * X[2])
F(X, I) = [dvdt(X, I), dudt(X)]

D, N = 2, 2 # number of dimensions, units
Nθ = 1000
G(X, K) = K * ([-1 1; 1 -1] * X[:, 1])
dt = 1e-3; T = 200; Nt = round(Int, T/dt)
trange = range(0, T, length=Nt)
alg = Tsit5();

In [4]:
@time Ts, ω, Xs = find_stable_periodic_solution(F, 0, D, Nθ, [2, 0.5], dt, alg, 1, 0.0, print_progress=false)
println("Ts=", Ts, " (sec) , ω=", ω, " (Hz)")

  6.473510 seconds (35.21 M allocations: 1.976 GiB, 6.98% gc time, 90.95% compilation time)
Ts=36.419000000000004 (sec) , ω=0.1725249267464671 (Hz)


In [15]:
# F, Imin, Imax, dI, D, Nθ, nothing, dt, alg, origin_val_idx, origin_thr
ωI, ζθI, ξθI, XsI = generalized_phase_sensitivity_func(F, -0.55, 0.55, 0.01, D, Nθ, nothing, dt, alg, 1, 0.0);

[1/3] Computing Xs(θ, I) and Z(θ, I)...100%|████████████| Time: 0:01:14


[2/3] Computing ζ(θ, I)...
[3/3] Computing ξ(θ, I)...


In [6]:
function coupled_func!(dX, X, p, t)
    g, κ = p
    Iext = g(X) # input
    for i in 1:size(dX)[1]
        dX[i, :] = κ[i] * F(X[i, :], Iext[i])
    end
end

coupled_func! (generic function with 1 method)

In [7]:
function coupled_original_system_fhn(N, D, Nt, dt, XsI, G, coupled_func!, initθ, κ, alg=Tsit5())
    X = zeros(Nt, N, D)
    initX = hcat([[XsI[j](θ, 0) for j in 1:D] for θ in initθ]...)'
    integrator = get_ode_integrator(coupled_func!, initX, dt, (G, κ), alg)
    for tt in 1:Nt
        x = copy(integrator.u)
        X[tt, :, :] = x # memory
        step!(integrator, dt, true)
    end
    return X 
end

coupled_original_system_fhn (generic function with 2 methods)

If `K>0.10`, IΘ cannot be computed because it's outside the range of external forces that can produce a stable periodic orbit.

In [8]:
alg = Tsit5()
NΘ = 50
K = 0.10 
κ = [1.0, 1.5]
initθ = [0, π/2];
g(X) = G(X, K)

g (generic function with 1 method)

In [16]:
QΘ = compute_QΘ(g, N, D, XsI, κ, ωI, NΘ);

Computing P(θ₁, θ₂)...100%|█████████████████████████████| Time: 0:00:24


In [17]:
Θrange = range(0, 2π, length=100);
QΘmap = QΘ[1](Θrange, Θrange);

In [18]:
PΘmap = zeros(100, 100);

for k in 1:100
    for l in 1:100
        Θ = [Θrange[k], Θrange[l]]
        PΘmap[k, l] = g(hcat([[XsI[j](Θ[i], QΘmap[k, l]) for j in 1:D] for i in 1:N]...)')[1] - QΘmap[k, l]
    end
end; 

### Run simulation

In [22]:
Xos = coupled_original_system_fhn(N, D, Nt, dt, XsI, g, coupled_func!, initθ, κ, alg);

200000×2×2 Array{Float64, 3}:
[:, :, 1] =
 -1.73472e-18   1.49926
  0.000942917   1.49893
  0.0018866     1.49859
  0.00283105    1.49826
  0.00377626    1.49793
  0.00472224    1.49761
  0.00566899    1.49728
  0.00661651    1.49695
  0.0075648     1.49662
  0.00851385    1.49629
  0.00946368    1.49597
  0.0104143     1.49564
  0.0113656     1.49532
  ⋮            
  1.36281      -0.244915
  1.36268      -0.246248
  1.36254      -0.247582
  1.36241      -0.248918
  1.36227      -0.250255
  1.36214      -0.251594
  1.36201      -0.252935
  1.36187      -0.254277
  1.36174      -0.25562
  1.3616       -0.256966
  1.36147      -0.258312
  1.36133      -0.25966

[:, :, 2] =
 0.0823908  1.32226
 0.0824416  1.32239
 0.0824924  1.32253
 0.0825433  1.32267
 0.0825943  1.3228
 0.0826454  1.32294
 0.0826965  1.32308
 0.0827477  1.32321
 0.0827989  1.32335
 0.0828503  1.32349
 0.0829017  1.32362
 0.0829532  1.32376
 0.0830048  1.3239
 ⋮          
 1.36764    1.68378
 1.36772    1.68367
 1.3678 

In [24]:
Xcpm, Θcpm = coupled_conventinal_phase_model(N, D, Nt, dt, XsI, g, ωI, ζθI, initθ, κ, alg);

LoadError: UndefVarError: D not defined

In [23]:
Xgpm, Θgpm = coupled_generalized_phase_model_PQ(N, D, Nt, dt, XsI, QΘ, ωI, ζθI, ξθI, initθ, κ, g, alg);

In [20]:
Irange = -0.55:0.01:0.55; 

In [21]:
figure(figsize=(12, 6), dpi=300)
ax3 = subplot2grid((2, 2), (0, 0), rowspan=1, colspan=1)
contour(Θrange, Θrange, QΘmap, 10, colors="white")
imshow(reverse(QΘmap, dims=1), extent=(0, 2pi, 0, 2pi))
xlabel(L"$\theta_1$"); ylabel(L"$\theta_2$");
xticks([0, pi, 2pi], ["0", L"$\pi$", L"$2\pi$"]);
yticks([0, pi, 2pi], ["0", L"$\pi$", L"$2\pi$"]);
divider3 = axes_grid1.make_axes_locatable(ax3)
cax3 = divider3.append_axes("right", size="5%", pad=0.1)
cbar3 = colorbar(cax=cax3)#, ticks=[-0.5, 0, 0.5])
cbar3.set_label(L"$Q_1\left(\theta_1, \theta_2\right)$")
ax3.text(-0.2, 1.2, "(a)", fontsize=16, transform=ax3.transAxes, fontweight="bold", va="top")

ax4 = subplot2grid((2, 2), (1, 0), rowspan=1, colspan=1)
contour(Θrange, Θrange, PΘmap, 10, colors="white")
imshow(reverse(PΘmap, dims=1), extent=(0, 2pi, 0, 2pi))
xlabel(L"$\theta_1$"); ylabel(L"$\theta_2$");
xticks([0, pi, 2pi], ["0", L"$\pi$", L"$2\pi$"]);
yticks([0, pi, 2pi], ["0", L"$\pi$", L"$2\pi$"]);
divider4 = axes_grid1.make_axes_locatable(ax4)
cax4 = divider4.append_axes("right", size="5%", pad=0.1)
cbar4 = colorbar(cax=cax4)#, ticks=[-0.5, 0, 0.5])
cbar4.set_label(L"$P_1\left(\theta_1, \theta_2\right)$")
ax4.text(-0.2, 1.2, "(b)", fontsize=16, transform=ax4.transAxes, fontweight="bold", va="top")

ax1 = subplot2grid((2, 2), (0, 1), rowspan=1, colspan=1)
# title("Conventinal phase model")
plot(trange, Xos[:, 1, 1], label="original system", color="tab:orange")
plot(trange, Xcpm[:, 1, 1], "--", label="phase eq. [conventional]", color="tab:red")
plot(trange, Xos[:, 2, 1], label=" ", color="tab:green")
plot(trange, Xcpm[:, 2, 1], "--", label=" ", color="tab:blue")
xlim(trange[1], trange[end]); ylim(ylim()[1]-0.3, ylim()[2]+2); ylabel(L"$v_1(t), v_2(t)$");
plt.gca().axes.xaxis.set_ticklabels([])
ax1.text(-0.15, 1.2, "(c)", fontsize=16, transform=ax1.transAxes, fontweight="bold", va="top")
legend(loc="upper right", ncol=2, columnspacing=0, labelspacing=0.1, frameon=false, markerfirst=false)

ax2 = subplot2grid((2, 2), (1, 1), rowspan=1, colspan=1)
plot(trange, Xos[:, 1, 1], label="original system", color="tab:orange")
plot(trange, Xgpm[:, 1, 1], "--", label="phase eq. [proposed]", color="tab:red")
plot(trange, Xos[:, 2, 1], label=" ", color="tab:green")
plot(trange, Xgpm[:, 2, 1], "--", label=" ", color="tab:blue")
xlim(trange[1], trange[end]); ylim(ylim()[1]-0.3, ylim()[2]+2);  xlabel(L"$t$"); ylabel(L"$v_1(t), v_2(t)$"); 
ax2.text(-0.15, 1.2, "(d)", fontsize=16, transform=ax2.transAxes, fontweight="bold", va="top")
legend(loc="upper right", ncol=2, columnspacing=0, labelspacing=0.1, frameon=false, markerfirst=false)
subplots_adjust(hspace=0.4, wspace=0.1)
#show()
#tight_layout()
savefig("fig3.svg")

LoadError: UndefVarError: Xcpm not defined